<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/02_DiffusionStokes_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorticity prediction and velocity (1d case)

(We will work in a cartesian coordinate system, x, y, z, with velocity u, v, w. )

A remarkably elegant version of the Navier-Stokes equations can be derived for an incompressible case.

$$
   \dot \Omega = \frac{\eta}{\rho} \nabla^2 \Omega  
$$

Further, we can make the problem 1 dimensional by integrating over the y and z direction. We will assume the z direction is bounded, and the y direction is infinitely long. Under these conditions, the incompressible condition in 1d means that the velocity u must be constant. For simplicty we will assume it is zero, without loss of generality. The w velocity is similarly zero. The velocity v need not be. These assumtions however, remove the need for non-linar advection, and we can write

$$
  \frac{\partial \zeta}{\partial t} = \frac{\eta}{\rho} \frac{\partial^2 \zeta}{\partial x^2}
$$

where $\zeta$ is the vorticity in the direction f the z coordinate.

$$
\zeta = \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}
$$


You have a case in which the initial vorticity is given by the function 
$$
\zeta(x,t=0) = A_1 sin(x - c_1) + A_2 sin(2x - c_2)
$$

With amplitudes $A_1 = 2$ and $A_2 = 1$. 



## Work tasks
1. Create a graph of vorticity 
2. Create function to approximate second derivative
3. Create a function to integrate in time
4. Produce time evolution to test analytic result.


*Learning goals*:
* Finite difference estimates of gradients and second derivatives
* Develop method for numerically solving diffusion equation
* Time dependent flow solutions
* Use of forward (Euler) time stepping
* Checking numerical and analytic results

 

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
print('All modules imported, ready to continue!')

The main component of this problem is developing an equation to calculate the second derivative. We wish to evaluate the second grid on a discrete grid between 0 and 2$\pi$, with steps $\Delta x$ indicated by index $i = 0, N-1$. (Note python has arrays startning at index 0

Using a finite difference method, we can obtain scheme with second order accuracy as:

$$
\frac{\partial^2 f}{\partial x^2} \approx
    \frac{f_{i+1} - 2 f_{i} + f_{i-1}}{4(\Delta x)^2}
$$

Create a function that performs this opperation. Recall we are working with periodic boundary conditions so we may "wrap arround" such that $f_{-1} = f_{N-1}$ and $f_{N} = f_{1}$. You may choose to do this with python array indices, or take a look at the numpy finction [numpy.roll()](https://numpy.org/doc/stable/reference/generated/numpy.roll.html).


In [ ]:
# Create a coordinate, which is periodix
npts = 20 
xvals = np.linspace(0,2*math.pi,npts)
dx = 2*math.pi/npts

rho = 1.
eta = 1.


# Define the initial vorticity
A1 = 1. 
A2 = 2.
c1 = math.pi*np.random.rand(1)
c2 = math.pi*np.random.rand(1)

vort = A1*np.sin(xvals - c1) + A2*np.sin(2*xvals - c2) 


Make a plot showing your initial vorticity: vorticity as a function of X

In [ ]:
# PLot!
p = plt.plot(xvals,vort)
plt.title('Vorticity')
plt.xlabel('X (radians)')
plt.ylabel('Vorticity (/s)')

We will need a function to compute the second derivative using finite differences. Let's do that here:

In [ ]:
def second_derivative(vort, dx):
    dfsq = np.zeros_like(vort)

    # This way using indices
    for i in range(npts):
      ip = i + 1
      if ip >= npts:       # special case for wrap around 
          ip = ip - npts

      im = i - 1          # special case for wrap around
      if im < 0:
          im = im + npts

      dfsq[i] = 0.25*(vort[ip] - 2*vort[i] + vort[im])/(dx*dx)


    return dfsq

# ALTURNATE: This way using "roll": same answer but faster and all in one line!
#def second_derivative(vort, dx):
#    return 0.25*(np.roll(vort,-1) -2*vort + np.roll(vort,+1))/(dx*dx)

Let's define a function to perform some number of time steps

In [ ]:
def forward_step(vort, nsteps, dtime):
    for n in range(nsteps):
        # evaluate the second derivative (your function!)
        dfsq = second_derivative(vort, dx)

        # formulate the tendency 
        dfdt = (eta/rho)*dfsq 

        # step forward
        vort = vort + dtime*dfdt
        #vort = dfdt
    
    return vort

Use your integration function to march forward in time to check the analytic result. Note, the time step must be small enough for a robust solution. It must be:

$$
\Delta t \lt \frac{(\Delta x)^2} {4 \eta}
$$



In [ ]:
dt_max = 0.25*dx*dx/eta
print("maximum allowed dtime is ",dt_max,"  seconds")

dtime = 0.5*dt_max
nsteps = 100

# Plot the initial function
p = plt.plot(xvals,vort)

# step forward by some number of steps, then plot again
vort = forward_step(vort,nsteps,dtime)
p = plt.plot(xvals,vort)

# step forward more steps, and plot again
vort = forward_step(vort,nsteps,dtime)
p = plt.plot(xvals,vort)

plt.title('Vorticity evolution of sines: nsteps='+str(nsteps))
plt.xlabel('X (radians)')
plt.ylabel('Vorticity (/s)')



#Results!

What was the predicted amplitue at this time? (Hint, it may be helpful to make a plot showing the analytic solution)



# The general case

In the case above, an analytic solution exists, and we can check out results. Notice that buy design the function as a sum of sine waves could be generalized:

$$
   \zeta(x,t=0) = \sum_{n} A_n sin(x - c_n)
$$

or more formally, a Fourier series. Since a fourier series can describe any arbitrary function, our method can evaluate any function, not just sines and cosines!

Try your clculation again, but start by defining a random function!

```
# random numbers!
vort[] = np.random.rand(npts)
```

Explain the result in the context of your findings above. Do your conclusions hold up?!





In [ ]:
# Set a bunch of intial random numbers
vort = A1*2*(np.random.rand(npts)-0.5)

# Plot the initial function
p = plt.plot(xvals,vort)

# step forward by some number of steps, then plot again
vort = forward_step(vort,nsteps,dtime)
p = plt.plot(xvals,vort)

# step forward more steps, and plot again
vort = forward_step(vort,nsteps,dtime)
p = plt.plot(xvals,vort)

plt.title('Vorticity evolution from random start: nsteps='+str(nsteps))
plt.xlabel('X (radians)')
plt.ylabel('Vorticity (/s)')



## Bonus!

The vorticity is related only to the velocity v. Develop a numerical method to calulate the velocity, v. 

*Task*: Create a time series graph of kinetic energy as a function of time. Does the diffusion of vorticity change the kinetic energy?









In [ ]:
#  vort = dv/dx, so integrate dv = vort*dx
def calc_velocity(vort, dx):
    # Need one integration constant, the mean velocity.
    vmean = 0

    # integration gives velocity at i+0.50, so need one extra point
    vplus = np.zeros(npts+1)
    for i in range(npts):
      vplus[i+1] = vplus[i] + dx*vort[i]

    # Assign the velocity at points "i" as the mean of those at i-0.5 and i+0.5
    velocity = 0.5*(vplus[0:npts] + vplus[1:npts+1])

    # Adjust the velocity so it has the required mean
    velocity = velocity - np.sum(velocity)/npts + vmean
    return velocity

def kinetic_energy(velocity):
    ke = rho*np.sum(velocity*velocity)/npts
    return ke

In [ ]:
# define a time series 
dtime = 0.5*dt_max
nsteps = 100

ke = np.zeros(nsteps)

# Start with random numbers
vort = A1*2*(np.random.rand(npts)-0.5)

for i in range(nsteps):
    vort = forward_step(vort,1,dtime)
    velocity = calc_velocity(vort,dx)
    ke[i] = kinetic_energy(velocity)

time = dtime*np.arange(nsteps)
p = plt.plot(time,ke)
plt.title("Kinetic energy series from random start")
plt.xlabel('time (s)')
plt.ylabel('K.E. (J/kg)')